<a href="https://colab.research.google.com/github/Lubraca/export-credit-risk-lab/blob/main/Notebooks/sce_indenization_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Configuring git

!git config --global user.name "Lubraca"
!git config --global user.email "lucas.casarin@hotmail.com"

In [2]:
# clone repository

!git clone https://github.com/Lubraca/export-credit-risk-lab

Cloning into 'export-credit-risk-lab'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 444.22 KiB | 3.80 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [3]:
%cd export-credit-risk-lab

/content/export-credit-risk-lab


Python Packages

In [4]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.4 MB/s eta 0:00:00


In [5]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import unidecode

Data Preparation

In [6]:
# import data from indenizations

# Dataset with insurance compensation for Brazilian Export Credit Insurance issued from ABGF available in the link below (there is a spreadsheet link in the bottom of the page):
# https://www.gov.br/mdic/pt-br/assuntos/camex/se-camex/sucex/financiamento/sce/indenizacoes-do-seguro-de-credito-a-exportacao

ic_df = pd.read_csv('/content/export-credit-risk-lab/pagamento_de_indenizacoes_Janeiro2026.csv', encoding='latin1')

# import data from BNDES

post_ship = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pos-embarque-bens.csv', encoding='latin1', sep=';')

eng_serv = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pos-embarque-servicos-de-engenharia.csv', encoding='latin1', sep=';')

pre_ship = pd.read_csv('/content/export-credit-risk-lab/operacoes-exportacao-operacoes-de-exportacao-pre-embarque.csv', encoding='latin1', sep=';')

### Helper Functions for Data Preparation and Transformation

This cell defines a collection of helper functions designed to clean, standardize, and transform the imported dataframes. These functions include:

- `standardize`: Standardizes column names and converts text columns to lowercase.
- `apply_corrections`: Applies custom corrections to specified columns using a mapping.
- `normalize_name`: Normalizes names by removing special characters, common legal suffixes (like S/A, Ltda), and sorting words.
- `add_norm_names`: Adds normalized name columns for 'exportador', 'importador', and 'beneficiario'.
- `parse_dates`: Converts specified columns to datetime objects.
- `add_contract_year`: Extracts the contract year from a 'certificado de garantia de cobertura' column.
- `convert_value`: Converts monetary values from string to float, handling 'USD' currency.
- `normalize_keyword`: Normalizes keywords for categorization.
- `add_object_category`: Categorizes export objects based on keywords.
- `set_categoricals`: Converts specified columns to categorical data types.
- `run_etl`: Orchestrates the execution of the above helper functions to perform a complete ETL (Extract, Transform, Load) process on a dataframe.

In [7]:
from typing import Dict, Optional, Callable

# Remove special characters from df column names and set them lowercase

def standardize(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [unidecode.unidecode(str(c)).lower().strip() for c in df.columns]
    text_cols = df.select_dtypes(include=["object"]).columns
    for c in text_cols:
        df[c] = df[c].astype("string").str.lower().str.strip()
    return df

# support function that removes any diacritic mark in every object that it is applied

def _norm_map_val(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    return unidecode.unidecode(str(x)).lower().strip()

# Function that maps texts before and after normalization with the function above "_norm_map_val"

def _norm_mapping(mapping: Dict[str, str]) -> Dict[str, str]:
    out: Dict[str, str] = {}
    for k, v in mapping.items():
        nk, nv = _norm_map_val(k), _norm_map_val(v)
        if pd.isna(nk) or pd.isna(nv):
            continue
        out[str(nk)] = str(nv)
    return out

# Check the columns value and analyse if them were treat my "_norm_mapping", if True is returns the normalized value
# otherwise, it keeps the value as it is

def apply_corrections(df: pd.DataFrame, corrections_by_column: Dict[str, Dict[str, str]]) -> pd.DataFrame:
    df = df.copy()
    for col, mapping in corrections_by_column.items():
        if col not in df.columns:
            continue
        df[col] = df[col].astype("string")
        mp = _norm_mapping(mapping)

        def _fix(x: object) -> object:
            nx = _norm_map_val(x)
            if pd.isna(nx):
                return pd.NA
            return mp.get(str(nx), x)

        df[col] = df[col].map(_fix)
    return df

# normalization of exporters, importers and beneficiary name

def normalize_name(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    s = unidecode.unidecode(str(x)).lower().strip()
# uses regular expressions to remove common Brazilian legal entity suffixes
    s = re.sub(r"\b(sa|s/a|ltda|epp|me)\b", "", s)
# remove any character that is not a lowercase letter (a-z) or a space
    s = re.sub(r"[^a-z ]", "", s)
#
    s = " ".join(sorted(s.split()))
    s = s.strip()
    return s if s else pd.NA


def add_norm_names(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    df = df.copy()
    cols = cols or ["exportador", "importador", "beneficiario"]
    for c in cols:
        if c in df.columns:
            df[f"{c}_norm"] = df[c].astype("string").map(normalize_name)
    return df


def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "autorizacao da indenizacao" in df.columns:
        df["autorizacao da indenizacao"] = pd.to_datetime(
            df["autorizacao da indenizacao"], format="%d-%m-%Y", errors="coerce"
        )
    if "data da inadimplencia" in df.columns:
        df["data da inadimplencia"] = pd.to_datetime(df["data da inadimplencia"], errors="coerce")
        df["ano de inadimplencia"] = df["data da inadimplencia"].dt.year
    return df


def add_contract_year(df: pd.DataFrame, contract_years_regex: str) -> pd.DataFrame:
    df = df.copy()
    col = "certificado de garantia de cobertura"
    if col in df.columns:
        df["ano do contrato"] = df[col].astype("string").str.extract(contract_years_regex).astype("float64")
    return df


def convert_value(df: pd.DataFrame, converter_valor: Callable[[object], object],
                  col: str = "valor total da indenizacao") -> pd.DataFrame:
    df = df.copy()
    if col in df.columns:
        df[col] = df[col].apply(converter_valor)
    return df


def normalize_keyword(x: object) -> object:
    if x is None or pd.isna(x):
        return pd.NA
    s = unidecode.unidecode(str(x)).lower().strip()
    s = re.sub(r"[^a-z0-9 ]", "", s)
    return s if s else pd.NA


def add_object_category(df: pd.DataFrame, categorize: Dict[str, list[str]],
                        source_col: str = "objeto da exportacao",
                        target_col: str = "objeto_categoria") -> pd.DataFrame:
    df = df.copy()
    if source_col not in df.columns:
        return df

    cat = {k: [str(normalize_keyword(v)) for v in vs] for k, vs in categorize.items()}

    def _classify(x: object) -> object:
        s = normalize_keyword(x)
        if pd.isna(s):
            return pd.NA
        s = str(s)
        for k, kws in cat.items():
            if any(kw in s for kw in kws):
                return k
        return "outros"

    df[target_col] = df[source_col].map(_classify)
    return df


def set_categoricals(df: pd.DataFrame, cols: Optional[list[str]] = None) -> pd.DataFrame:
    df = df.copy()
    cols = cols or [
        "exportador", "importador", "beneficiario",
        "exportador_norm", "importador_norm", "beneficiario_norm",
        "pais", "objeto da exportacao", "financiador",
        "modalidade de financiamento", "natureza do risco",
        "forma de pagamento do premio",
        "objeto_categoria",
    ]
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype("category")
    return df


def run_etl(
    df: pd.DataFrame,
    contract_years_regex: str,
    converter_valor: Callable[[object], object],
    categorize: Optional[Dict[str, list[str]]] = None
) -> pd.DataFrame:
    df2 = standardize(df)
    df2 = add_norm_names(df2)
    df2 = parse_dates(df2)
    df2 = add_contract_year(df2, contract_years_regex)
    df2 = convert_value(df2, converter_valor)
    if categorize:
        df2 = add_object_category(df2, categorize)
    df2 = set_categoricals(df2)
    return df2


In [ ]:
# Create year time variable from the contract order (assumimng that the year present in the order number represents the contrant year)

years = np.arange(start = 2000, stop = 2026)
contract_years = "|".join(map(str,years))

# standardization of values ​​in USD
def converter_valor(valor):
    if isinstance(valor, str) and "USD" in valor:
        valor = valor.replace("USD", "").replace(".", "").replace(",", ".").strip()
        return float(valor)
    return valor

categorize = {
    'Aeronaves': ['Aeronaves','aeronave', 'Aerovane' 'aeronaves', 'EMB', 'Avião', 'avião', 'Boeing', 'Airbus', 'Embraer', 'Helicóptero', 'helicoptero', 'Helicópteros', 'helicopteros'],
    'Construção Civil': ['Elementos','Estrutura','construção', 'obras', 'aeroporto', 'porto', 'infraestrutura', 'metrô','edificações'],
    'Agronegócio': ['agrícola', 'colheitadeiras', 'tratores', 'arroz', 'fertilizantes', 'plantio', 'proteína','cana'],
    'Equipamentos': ['Equpamentos','Equipamentos','Sistema','Máquina', 'Máquinas','maquina','máquinas', 'equipamentos', 'Prensas', 'prensas', 'sistemas', 'hidráulicos','Sonda','Extrator', 'Bens'],
    'Produtos': ['Produtos','produtos','químicos','prensas','Lâminas', 'lâminas', 'cremes', 'pinturas', 'Impressora','produção','Móveis', 'Módulos', 'Elevadores','Balanças'],
    'Veículos': ['Colheitadeiras','Escavadeiras','Caminhões','caminhões', 'veículos', 'retroescavadeiras', 'escavadeiras', "caminhão", "ônibus", "carros", "veículo"],
}

contract_years_regex=r"\b(200\d|201\d|202[0-5])\b"

In [ ]:
df_clean = run_etl(
    df=ic_df,
    contract_years_regex=contract_years_regex,
    converter_valor=converter_valor,
    categorize=categorize,   # optional, remove if not needed
)

In [ ]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 27 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   certificado de garantia de cobertura          1106 non-null   string        
 1   data da inadimplencia                         208 non-null    datetime64[ns]
 2   autorizacao da indenizacao                    0 non-null      datetime64[ns]
 3   valor total da indenizacao                    1106 non-null   object        
 4   exportador                                    1106 non-null   category      
 5   importador                                    1106 non-null   category      
 6   beneficiario                                  1106 non-null   category      
 7   pais                                          1106 non-null   category      
 8   objeto da exportacao                          1106 non-null   catego

In [ ]:
df_clean[["exportador", "exportador_norm"]].head()


,exportador,exportador_norm
0,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
1,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
2,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
3,construtora norberto odebrecht s.a.,construtora norberto odebrecht sa
4,alpha química ltda.,alpha quimica


In [ ]:
df_clean["objeto_categoria"].value_counts(dropna=False)


,count
objeto_categoria,
Agronegócio,341
Construção Civil,313
Equipamentos,244
Aeronaves,117
Veículos,45
Produtos,35
outros,11


In [ ]:
df["ano do contrato"].value_counts(dropna=False).sort_index()

,count
ano do contrato,
2008.0,161
2009.0,76
2010.0,352
2011.0,92
2012.0,143
2013.0,49
2014.0,118
2015.0,32
2016.0,40
